# Welcome to ProgPy's Linear Model Example

The goal of this notebook is to instruct users on how to use ProgPy Model LinearModel.

This example shows the use of the LinearModel class, a subclass of PrognosticsModel for models that can be described as a linear time series, which can be defined by the following equations:


$$
\frac{dx}{dt} = Ax + Bu + E
$$

$$
z = Cx + D
$$

$$
es = Fx + G
$$

$x$ is `state`, $u$ is `input`, $z$ is `output`, and $es$ is `event state`

Furthermore, Linear Models must inherit from this class and define the following properties:
* $A$: 2-D np.array[float], dimensions: n_states x n_states
* $B$: 2-D np.array[float], optional (zeros by default), dimensions: n_states x n_inputs
* $C$: 2-D np.array[float], dimensions: n_outputs x n_states
* $D$: 1-D np.array[float], optional (zeros by default), dimensions: n_outputs x 1
* $E$: 1-D np.array[float], optional (zeros by default), dimensions: n_states x 1
* $F$: 2-D np.array[float], dimensions: n_es x n_states
* $G$: 1-D np.array[float], optional (zeros by default), dimensions: n_es x 1
* __inputs__:  list[str] - `input` keys
* __states__:  list[str] - `state` keys
* __outputs__: list[str] - `output` keys
* __events__:  list[str] - `event` keys

We will now utilize our LinearModel to model the classical physics problem throwing an object into the air! We can create a subclass of LinearModel which will be used to simulate an object thrown, which we will call the ThrownObject Class.


First, some definitions for our Model!

    Events (2)
        | falling: The object is falling
        | impact: The object has hit the ground

    Inputs/Loading: (0)

    States: (2)
        | x: Position in space (m)
        | v: Velocity in space (m/s)

    Outputs/Measurements: (1)
        | x: Position in space (m)

Now, for our keyword arguments:

* `g : Optional, float`
  * Acceleration due to gravity (m/s^2). Default is 9.81 m/s^2 (standard gravity)
* `thrower_height : Optional, float`
  * Height of the thrower (m). Default is 1.83 m
* `throwing_speed : Optional, float`
  * Speed at which the ball is thrown (m/s). Default is 40 m/s

With our definitions, we can now create the ThrownObject Model.

First, we need to import the necessary packages.

In [ ]:
import numpy as np
from prog_models import LinearModel

Now we'll define some features of ThrownObject such that it models after a LinearModel. Recall that all LinearModels follow a set of core equations and require some specific properties (see above). In the next step, we'll define our inputs, states, outputs, and events, along with the $A$, $C$, $E$, and $F$ values.

In [ ]:
class ThrownObject(LinearModel):
    events = ['impact']
    inputs = []  
    states = ['x', 'v']
    outputs = ['x']
    
    A = np.array([[0, 1], [0, 0]])
    C = np.array([[1, 0]])
    E = np.array([[0], [-9.81]])
    F = None

Next, we'll define some default parameters for our ThrownObject model.

In [ ]:
class ThrownObject(ThrownObject):  # Continue the ThrownObject class
    default_parameters = {
        'thrower_height': 1.83,
        'throwing_speed': 40,
        'g': -9.81
    }

In the following cells, we'll define some class functions necessary to perform prognostics on the model.

The `initialize()` function sets the states to their corresponding values. Since we have defined the `x` value for our ThrownObject model to represent position in space, our initial value would be the thrower_height parameter.

In [ ]:
class ThrownObject(ThrownObject):
    def initialize(self, u=None, z=None):
        return self.StateContainer({
            'x': self.parameters['thrower_height'],
            'v': self.parameters['throwing_speed']
            })

For our `threshold_met()`, we have defined the function to return a boolean function when our thrown object has met the initial state of 'impact', or in other words, has hit the ground. This occurs when our height is less than or equal to 0, and our speed is less than 0!

In [ ]:
class ThrownObject(ThrownObject):
    def threshold_met(self, x):
        return {
            'falling': x['v'] < 0,
            'impact': x['x'] <= 0
        }

Finally, for our `event_state()`, we have defined our events to be when our object has hit the ground, or has made 'impact'. The `event_state()` function specifically returns the current state of our object at any given point of it's journey through the air!

In [ ]:
class ThrownObject(ThrownObject):
    def event_state(self, x): 
        x_max = x['x'] + np.square(x['v'])/(-self.parameters['g']*2)
        return {
            'falling': np.maximum(x['v']/self.parameters['throwing_speed'],0),
            'impact': np.maximum(x['x']/x_max,0) if x['v'] < 0 else 1
        }

With these functions created, we can now run our ThrownObject Model!

In this example, we will initialize our ThrownObject as `m`, and we'll define our `future_loading()` function (more information about future loading: https://nasa.github.io/progpy/prog_models_guide.html#future-loading). We'll use the `simulate_to_threshold()` function (more information about simulation: https://nasa.github.io/progpy/prog_models_guide.html#simulation) to simulate the movement of the thrown object in air.

In [ ]:
m = ThrownObject()
def future_loading(t, x=None):
    return m.InputContainer({})
save = m.simulate_to_threshold(future_loading, print = True, save_freq=1, threshold_keys='impact', dt=0.1)

We'll also demonstrate how this looks plotted on a graph.

In [ ]:
import matplotlib.pyplot as plt
save.outputs.plot(title='generated model')
plt.show()